# Sales: Recommender System (collaborative filtering)

*Created by: Anna Sukhareva, data scientist  
Antwerpen, Belgium  
Date: November 13, 2019  
Contacts: anna@linefeed.be*  
***

### Table of Contents
Stage 1 : Business Understanding  
Stage 2 : Analytic Approach  
Stage 3 : Data Requirements  
Stage 4 : Data Collection  
Stage 5 : Data Understanding & Preparation  
Stage 6 : Modeling   
***

### Stage 1 : Business Understanding
**Problem:**  
Our client is a video streaming company. Most of client's revenues come from a fixed-rate recurring billing model subscription. One of the biggest business tasks - customer retention. To do so, the company needs to continuously calibrate suggested content  to the preferences of the user.  

Some of interesting statistics:
+ According to a Netflix executives, the video streaming service’s recommendation system saves the company around $1 billion each year,   
+ According to McKinsey, 75 percent of what users watch on Netflix come from product recommendations.   

**Question:**  
How find out a product to recommend for a customer which he highly likely to buy?

### Stage 2 : Analytic Approach
Collaborative filtering (user-to-user) recommender model.

### Stage 3 : Data Requirements
**Data content:** To answer the question we need data about **movies** (title, year of release, genre, cast, etc) and **ratings** (user's ratings to some of those movies)   
**Data formats:** CSV format    
**Data sources:** Dataset acquired from [GroupLens](http://grouplens.org/datasets/movielens/)

### Stage 4 :  Data Collection
Importing libraries:

Dataset acquired from [GroupLens](http://grouplens.org/datasets/movielens/).   
Downloading using `!wget` :

I've download and save data in my IBM Cloud, access with SQL [link](https://cloud.ibm.com/objectstorage/crn%3Av1%3Abluemix%3Apublic%3Acloud-object-storage%3Aglobal%3Aa%2Fc0dece5aae5d40f4a1d377abc34ced91%3A68c548fa-a77c-4d98-a67c-eff26fee7184%3A%3A?bucket=recommendersystemlab&bucketRegion=eu-de&endpoint=s3.eu-de.cloud-object-storage.appdomain.cloud&paneId=bucket_overview)



In [1]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2019-11-11 13:09:26--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.193
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  28.2MB/s    in 5.7s    

2019-11-11 13:09:32 (27.0 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


In [3]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Creating dataframes:

In [4]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

###  Stage 5 : Data Understanding & Preparation  

In [5]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


**Movies dataframe:**  
Data manipulation:  removing the year from the title column to a separate column and dropping the 'genres' column:

In [6]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False) #removing the parentheses
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '') #Removing the years
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip()) #Applying the strip function to get rid of any ending whitespace characters
movies_df = movies_df.drop('genres', 1) #we won't need it for this particular recommendation system
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


<br>

**Ratings dataframe:**

In [9]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


Data manipulation: dropping timestamp column to save on memory.

In [7]:
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


### Stage 6 : Modeling  
'Collaborative' or 'User-User' filtering

This technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. 
To find similar users we use Pearson Correlation Function

The process for creating a User Based recommendation system is as follows:
1. Select a user with the movies the user has watched
2. Based on his rating to movies, find the top X neighbours 
3. Get the watched movie record of the user for each neighbour.
4. Calculate a similarity score using some formula
5. Recommend the items with the highest score

Note: Follow the naming convention in movies titles: capital letters, if a movie starts with a "The" , like "The Matrix" then write it in like this: 'Matrix, The' .  

**Creating an input user:**

In [8]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


**Add movieId to input user** : With the input complete, we extract the input movies's ID's from the movies dataframe and add them into it by filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [10]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())] #filtering out the movies
inputMovies = pd.merge(inputId, inputMovies) 
inputMovies = inputMovies.drop('year', 1) 
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


**Finding similar users** (who reviewed the same movies):


In [11]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0


In [12]:
userSubset.shape

(196623, 3)

Group up by user ID:

In [14]:
userSubsetGroup = userSubset.groupby(['userId'])

Take a random user, f.e. userID 1130

In [15]:
userSubsetGroup.get_group(1130)

,userId,movieId,rating
104167,1130,1,0.5
104168,1130,2,4.0
104214,1130,296,4.0
104363,1130,1274,4.5
104443,1130,1968,4.5


Sort  groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.

In [16]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

Checking the first user:

In [18]:
userSubsetGroup[0:3]

[(75,       userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0), (106,       userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5), (686,        userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0)]

**Computing similarity of users to input user**  
Comparing all users to the input user and find the one that is most similar, using the Pearson Correlation Coefficient, which is used to measure the strength of a linear association between two variables. Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below. 


![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation. In our case, a 1 means that the two users have similar tastes while a -1 means the opposite. We select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.

In [19]:
userSubsetGroup = userSubsetGroup[0:100]

Calculating the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient:

In [17]:
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #sort the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    nRatings = len(group) #Get the N for the formula
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())] #Get the review scores for the movies that they both have in common
    tempRatingList = temp_df['rating'].tolist() #store them in a temporary buffer variable in a list format to facilitate future calculations
    tempGroupList = group['rating'].tolist() #put the current user group reviews in a list format
    #calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [ ]:
pearsonCorrelationDict.items()

In [22]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.827278,75
1,0.586009,106
2,0.832050,686
3,0.576557,815
4,0.943456,1040


Getting top 50 similar users:

In [23]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
64,0.961678,12325
34,0.961538,6207
55,0.961538,10707
67,0.960769,13053
4,0.943456,1040


**Rating of selected users to all movies** : We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our __pearsonDF__ from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [24]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,0.961678,12325,1,3.5
1,0.961678,12325,2,1.5
2,0.961678,12325,3,3.0
3,0.961678,12325,5,0.5
4,0.961678,12325,6,2.5


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights. We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns: It shows the idea of all similar users to candidate movies for the input user:

In [25]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.961678,12325,1,3.5,3.365874
1,0.961678,12325,2,1.5,1.442517
2,0.961678,12325,3,3.0,2.885035
3,0.961678,12325,5,0.5,0.480839
4,0.961678,12325,6,2.5,2.404196


In [26]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,38.376281,140.800834
2,38.376281,96.656745
3,10.253981,27.254477
4,0.929294,2.787882
5,11.723262,27.151751


In [27]:
recommendation_df = pd.DataFrame() #empty dataframe
#taking the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.668955,1
2,2.518658,2
3,2.657941,3
4,3.000000,4
5,2.316058,5


Sort it and see the top 20 movies that the algorithm recommended

In [28]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
5073,5.0,5073
3329,5.0,3329
2284,5.0,2284
26801,5.0,26801
6776,5.0,6776
6672,5.0,6672
3759,5.0,3759
3769,5.0,3769
3775,5.0,3775


In [29]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
2200,2284,Bandit Queen,1994
3243,3329,"Year My Voice Broke, The",1987
3669,3759,Fun and Fancy Free,1947
3679,3769,Thunderbolt and Lightfoot,1974
3685,3775,Make Mine Music,1946
4978,5073,"Son's Room, The (Stanza del figlio, La)",2001
6563,6672,War Photographer,2001
6667,6776,Lagaan: Once Upon a Time in India,2001
9064,26801,Dragon Inn (Sun lung moon hak chan),1992
18106,90531,Shame,2011
